# **Модуль Б**. Разработка модели машинного обучения

## Импортирование библиотек

In [100]:
# модули для работы с моделью
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

import pandas as pd

from IPython.display import Audio

from Model.tokenizer import Tokenizer

import numpy as np

## Загрузка данных

In [101]:
# путь к обработанным данным
df_path = '../Module1/Dataset/dataset.h5'
# загружаем датасета
df = pd.read_hdf(df_path, key='df')
df.head()

,path,sentence,record_duration,sampling_rate,array,text_length,snr,rms_dB,tokens,token_ids,mel_spec
0,Data/cv_corpus/cv-corpus-21.0-delta-2025-03-14...,"Абай был не только талантливым поэтом, но и уч...",5.04,16000,"[-1.0186341e-10, 6.91216e-11, -8.0035534e-11, ...",51,29.652618,-21.792862,"[абай, _, был, _, не, _, только, _, талантливы...","[5165, 3734, 5298, 3734, 2808, 3734, 932, 3734...","[[-13.040601, -10.495326, -7.282387, -6.365622..."
1,Data/cv_corpus/cv-corpus-21.0-delta-2025-03-14...,Гибкая кошка легко взбирается на высокое дерево.,5.65,16000,"[2.910383e-11, -1.1641532e-10, 1.9645086e-10, ...",48,27.797586,-23.426945,"[гибкая, _, кошка, _, легко, _, взбирается, _,...","[3706, 3734, 2332, 3734, 1692, 3734, 4548, 373...","[[-3.2210479, -4.417929, -6.569694, -5.2323184..."
2,Data/cv_corpus/cv-corpus-21.0-delta-2025-03-14...,Сделать нужно гораздо больше.,3.64,16000,"[-1.3096724e-10, -5.820766e-11, -1.4551915e-10...",29,27.732599,-24.547728,"[сделать, _, нужно, _, гораздо, _, больше]","[4891, 3734, 6461, 3734, 1353, 3734, 395, 6674...","[[-3.1968756, -3.3615067, -3.878891, -5.443349..."
3,Data/cv_corpus/cv-corpus-21.0-delta-2025-03-14...,"Эти ноты, сливаясь воедино, образуют симфонию,...",6.62,16000,"[8.731149e-11, -6.91216e-11, 0.0, -8.0035534e-...",77,23.927074,-25.052473,"[эти, _, ноты, _, сливаясь, _, воедино, _, обр...","[243, 3734, 5291, 3734, 3835, 3734, 3604, 3734...","[[-12.80517, -11.202806, -9.942759, -6.5413194..."
4,Data/cv_corpus/cv-corpus-21.0-delta-2025-03-14...,Квадрат гипотенузы равен сумме квадратов катетов.,5.83,16000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",49,35.466934,-17.979994,"[квадрат, _, гипотенузы, _, равен, _, сумме, _...","[5016, 3734, 3892, 3734, 4864, 3734, 4855, 373...","[[-5.968595, -4.914544, -4.8844056, -4.73808, ..."


In [102]:
text_example = df['sentence'][42]
text_example

'Европейский союз будет оставаться наблюдателем в Генеральной Ассамблее.'

**Класс с моделью**

In [113]:
class TTSTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=256):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, nhead=4),
            num_layers=3
        )
        self.decoder = nn.Sequential(
            nn.Linear(d_model, 512),
            nn.ReLU(),
            nn.Linear(512, 429)  # n_mels = 429
        )
    
    def forward(self, tokens):
        x = self.embedding(tokens)
        x = self.encoder(x)
        return self.decoder(x)

In [104]:
class TTSDataset(Dataset):
    def __init__(self, tokenized_texts, mel_spectrograms, 
                 pad_token_id=0, mel_pad_value=-100):
        """
        Улучшенная версия датасета с тщательной проверкой размерностей
        
        Args:
            tokenized_texts: список списков int (токены)
            mel_spectrograms: список np.array формы (time, n_mels)
            pad_token_id: ID для паддинга текста
            mel_pad_value: значение для паддинга спектрограмм
        """
        self.tokenized_texts = tokenized_texts
        self.mel_spectrograms = mel_spectrograms
        self.pad_token_id = pad_token_id
        self.mel_pad_value = mel_pad_value
        
        # Проверка согласованности данных
        self._validate_data()
        
        # Запоминаем размерность мел-признаков
        self.n_mels = mel_spectrograms[0].shape[1]

    def _validate_data(self):
        assert len(self.tokenized_texts) == len(self.mel_spectrograms), \
            f"Несовпадение количества текстов ({len(self.tokenized_texts)}) и спектрограмм ({len(self.mel_spectrograms)})"
        
        # Проверяем что все спектрограммы 2D и имеют одинаковое n_mels
        n_mels = self.mel_spectrograms[0].shape[1]
        for i, mel in enumerate(self.mel_spectrograms):
            assert mel.ndim == 2, f"Спектрограмма {i} должна быть 2D, но имеет {mel.ndim} измерений"
            assert mel.shape[1] == n_mels, \
                f"Несовпадение n_mels: ожидается {n_mels}, получено {mel.shape[1]} в спектрограмме {i}"

    def __len__(self):
        return len(self.tokenized_texts)

    def __getitem__(self, idx):
        # Возвращаем тензоры напрямую
        return (
            torch.LongTensor(self.tokenized_texts[idx]),
            torch.FloatTensor(self.mel_spectrograms[idx])
        )

In [105]:
def collate_fn(batch):
    """Улучшенная функция для создания батчей с паддингом"""
    tokens, mels = zip(*batch)
    
    # 1. Паддинг токенов
    tokens_padded = pad_sequence(
        tokens, 
        batch_first=True, 
        padding_value=0
    )
    
    # 2. Паддинг спектрограмм
    # Находим максимальную длину в батче
    max_len = max(mel.size(0) for mel in mels)
    n_mels = mels[0].size(1)
    
    # Создаем большой тензор с паддингом
    mels_padded = torch.full(
        (len(mels), max_len, n_mels),
        fill_value=-100,
        dtype=torch.float32
    )
    
    # Заполняем данные без ошибок размерностей
    for i, mel in enumerate(mels):
        actual_len = mel.size(0)
        mels_padded[i, :actual_len, :] = mel  # Точно соответствуем размерностям
    
    return tokens_padded, mels_padded

In [106]:
tokenized_texts = df['token_ids']
mel_spectrograms = df['mel_spec']


In [107]:
# Создаем и проверяем датасет
try:
    dataset = TTSDataset(tokenized_texts, mel_spectrograms)
    print("Датасет создан успешно!")
    
    # Проверяем один пример
    tokens, mel = dataset[1]
    print("\nПример 1:")
    print(f"Токены: {tokens.shape} (макс: {tokens.max().item()})")
    print(f"Спектрограмма: {mel.shape} (макс: {mel.max().item():.2f})")
    
    # Создаем DataLoader с обработкой ошибок
    dataloader = DataLoader(
        dataset,
        batch_size=2,
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=0  # Для лучшей отладки
    )
    
    # Тестируем батчи
    for batch_idx, (tokens_batch, mels_batch) in enumerate(dataloader):
        print(f"\nБатч {batch_idx}:")
        print(f"Токены: {tokens_batch.shape}")
        print(f"Спектрограммы: {mels_batch.shape}")
        print(f"Реальные длины: {[mel.size(0) for mel in mels]}")
        break
        
except Exception as e:
    print(f"Ошибка при создании датасета: {str(e)}")
    raise

Датасет создан успешно!

Пример 1:
Токены: torch.Size([27]) (макс: 6674)
Спектрограмма: torch.Size([80, 429]) (макс: 6.52)

Батч 0:
Токены: torch.Size([2, 27])
Спектрограммы: torch.Size([2, 80, 429])
Ошибка при создании датасета: name 'mels' is not defined


NameError: name 'mels' is not defined

In [108]:
# Проверяем один пример
tokens, mel = dataset[0]
print("Пример 0:")
print(f"Токены: {tokens} (длина: {len(tokens)})")
print(f"Спектрограмма: {mel.shape}")

Пример 0:
Токены: tensor([5165, 3734, 5298, 3734, 2808, 3734,  932, 3734,  460, 3734, 5910, 3734,
        4008, 3734, 4773, 3734, 1197, 6674, 6674, 6674, 6674, 6674, 6674, 6674,
        6674, 6674, 6674]) (длина: 27)
Спектрограмма: torch.Size([80, 429])


In [109]:
# Создаем DataLoader
dataloader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=collate_fn
)


In [110]:
# Проверяем батч
for batch_idx, (tokens_batch, mels_batch) in enumerate(dataloader):
    print(f"\nБатч {batch_idx}:")
    print(f"Токены: {tokens_batch.shape}")
    print(f"Спектрограммы: {mels_batch.shape}")
    break


Батч 0:
Токены: torch.Size([2, 27])
Спектрограммы: torch.Size([2, 80, 429])


In [111]:
tokenizer = Tokenizer()

/home/user/Chemp/Speech-synthesis/Module2


In [112]:
model = TTSTransformer(vocab_size=len(tokenizer.text_to_ids_voc))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(10):
    for tokens, mels in dataloader:
        optimizer.zero_grad()
        pred_mels = model(tokens)
        loss = criterion(pred_mels, mels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch}, Loss: {loss.item()}")

/home/user/Chemp/.venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/home/user/Chemp/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([2, 80, 429])) that is different to the input size (torch.Size([2, 27, 80])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (80) must match the size of tensor b (429) at non-singleton dimension 2